## Binary classification neural network from scratch

In [30]:
import os
import numpy as np
import torch
import requests
import tarfile

from PIL import Image
from torch import tensor
from torch.utils.data import DataLoader
import torchvision.datasets
from pathlib import Path
from dataclasses import dataclass

#### MetaParameters
- keeping track of all global variables: Batch size, learning rate, number of epochs

In [33]:
### ADJUST METAPARAMETERS HERE
@dataclass
class MetaParameters:
    batch_size: int
    learning_rate: float
    n_epochs: int
    
mp = MetaParameters(64, 1., 8)
mp

MetaParameters(batch_size=64, learning_rate=1.0, n_epochs=8)

In [34]:
dev = torch.device('cuda')

In [35]:
def ls(path: Path):
    return [p for p in path.glob('./*')]

In [37]:
path = Path('/home/c/projects/NN-from-scratch/mnist_png')
ls(path), path.name

([Path('/home/c/projects/NN-from-scratch/mnist_png/testing'),
  Path('/home/c/projects/NN-from-scratch/mnist_png/training')],
 'mnist_png')

In [ ]:
# Note: p represents purpose between train and valid
data = {
    p: {
        number: torch.stack([tensor(np.array(Image.open(o))).to(dev) 
                for o in ls((path/f'{p}/{number}'))]).float()/255 for number in ('3', '7')
    } for p in ('train', 'valid')
}

In [ ]:
x = {
    p: torch.cat((data[p]['3'], data[p]['7'])).view(-1, 28*28)
                for p in ('train', 'valid')
}
y = {
    p: tensor([1] * len(data[p]['3']) + [0] * len(data[p]['7']), device=dev).unsqueeze(1)
                for p in ('train', 'valid')
}

dsets = {p: list(zip(x[p], y[p])) for p in ('train', 'valid')}

In [ ]:
dls = {p: DataLoader(dsets[p], batch_size=mp.batch_size, shuffle=True) for p in ('train', 'valid')}

In [ ]:
def model(x, w, b):
    return x@w + b

In [ ]:
def mnist_loss(predictions, targets) -> float:
    predictions = predictions.sigmoid()
    return torch.where(targets == 0, predictions, 1 - predictions).mean()

In [ ]:
def init_params(shape):
    return torch.randn(shape, requires_grad=True, dtype=torch.float, device=dev)

In [ ]:
def train(dl, params):
    for x, y in dl:
        preds = model(x, weights, bias)
        loss = mnist_loss(preds, y)
        loss.backward()
        for p in params:
            p.data -= p.grad * mp.learning_rate
            p.grad.zero_()

In [ ]:
weights = init_params((28*28, 1))
bias = init_params(1)

In [ ]:
def batch_accuracy(preds, trgts):
    preds = preds.sigmoid()
    return ((preds > .5) == trgts).float().mean()

def accuracy(valid_dl, weights, bias):
    accs = [batch_accuracy(model(xb, weights, bias), yb) for xb, yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 7)

In [ ]:
params = weights, bias
for i in range(mp.n_epochs):
    #train_epoch(train_dl, model, mp.learning_rate, params)
    #print(validate_epoch(model))
    train(dls['train'], params)
    print(accuracy(dls['valid'], weights, bias))

0.9598705
0.9690574
0.9696361
0.9731445
0.9730541
0.9750072
0.9754051
0.9758934
